In [1]:
import tensorflow as tf

import tf_encrypted as tfe
from tf_encrypted.keras import backend as KE
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD

from common import DataOwner

import sys
sys.path.append('../')
from utils import print_in_notebook

/anaconda3/envs/py35_int64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_features = 10
training_set_size = 2000
test_set_size = 100
batch_size = 100
steps_per_epoch = (training_set_size // batch_size)
epochs = 20

In [3]:
data_owner = DataOwner('data-owner',
                       num_features,
                       training_set_size,
                       test_set_size,
                       batch_size)

x_train, y_train = data_owner.provide_training_data()
x_test, y_test = data_owner.provide_testing_data()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
@tfe.local_computation('data-owner')
def compute_accuracy(y_pred, y_test):
    correct_prediction = tf.equal(tf.round(y_pred), y_test)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print_op = print_in_notebook(accuracy)
    return print_op

## Define Model

In [5]:
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, num_features]))
model.add(tfe.keras.layers.Activation('sigmoid'))

In [6]:
sess = KE.get_session()
sess.run([data_owner.initializer])

model.compile(optimizer=SGD(lr=0.01),
              loss=BinaryCrossentropy())

# Train
model.fit(x_train,
          y_train,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch)

# evaluate
y_pred = model(x_test)
sess.run(compute_accuracy(y_pred, y_test))

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2', 'data-owner']


Epoch 1/20
2000/2000 [==============================] - 2s 1ms/step - loss: 0.6981
Epoch 2/20
2000/2000 [==============================] - 0s 76us/step - loss: 0.6439
Epoch 3/20
2000/2000 [==============================] - 0s 76us/step - loss: 0.6053
Epoch 4/20
2000/2000 [==============================] - 0s 74us/step - loss: 0.5617
Epoch 5/20
2000/2000 [==============================] - 0s 79us/step - loss: 0.5327
Epoch 6/20
2000/2000 [==============================] - 0s 80us/step - loss: 0.5118
Epoch 7/20
2000/2000 [==============================] - 0s 76us/step - loss: 0.4894
Epoch 8/20
2000/2000 [==============================] - 0s 75us/step - loss: 0.4690
Epoch 9/20
2000/2000 [==============================] - 0s 75us/step - loss: 0.4521
Epoch 10/20
2000/2000 [==============================] - 0s 76us/step - loss: 0.4355
Epoch 11/20
2000/2000 [==============================] - 0s 75us/step - loss: 0.4251
Epoch 12/20
2000/2000 [==============================] - 0s 74us/step - los

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


0.95


In [7]:
type(x_train)

tf_encrypted.protocol.pond.pond.PondPrivateTensor